#### Instructions:  
1. Libraries allowed: **Python basic libraries, numpy, pandas, scikit-learn (only for data processing), pytorch, and ClearML.**
2. Show all outputs.
3. Submit jupyter notebook and a pdf export of the notebook. Check canvas for detail instructions for the report. 
4. Below are the questions/steps that you need to answer. Add as many cells as needed. 

## Task 2: Finetuning a pretrained NN
Do transfer learning with ResNet18 and compare peforamnce with the hyperparamter-tuned network.

In [38]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [39]:
data_dir = r"D:\Semester 3\Deep Learning\Assignments\Project1\Dataset\organized_dataset"

In [40]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [41]:
datasets = {
    'train': datasets.ImageFolder(root=f"{data_dir}/train", transform=data_transforms['train']),
    'val': datasets.ImageFolder(root=f"{data_dir}/val", transform=data_transforms['val']),
    'test': datasets.ImageFolder(root=f"{data_dir}/test", transform=data_transforms['test'])    
}

# Create dataloaders
dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(datasets['val'], batch_size=32, shuffle=False, num_workers=4),
    'test': DataLoader(datasets['test'], batch_size=32, shuffle=True, num_workers=4)
}

In [15]:
len(datasets['train'].classes)

7

In [12]:
model = models.resnet18(pretrained=True)
num_classes = len(datasets['train'].classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(datasets[phase])
        epoch_acc = running_corrects.double() / len(datasets[phase])

        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

torch.save(model.state_dict(), "resnet18_finetuned.pth")

Epoch 1/10
----------
Train Loss: 0.4774 Acc: 0.8348
Val Loss: 0.7619 Acc: 0.7929
Epoch 2/10
----------
Train Loss: 0.3486 Acc: 0.8991
Val Loss: 0.5386 Acc: 0.8143
Epoch 3/10
----------
Train Loss: 0.1892 Acc: 0.9473
Val Loss: 0.2407 Acc: 0.9143
Epoch 4/10
----------
Train Loss: 0.1079 Acc: 0.9616
Val Loss: 0.0564 Acc: 0.9929
Epoch 5/10
----------
Train Loss: 0.0680 Acc: 0.9795
Val Loss: 0.1849 Acc: 0.9357
Epoch 6/10
----------
Train Loss: 0.0292 Acc: 0.9938
Val Loss: 0.0903 Acc: 0.9714
Epoch 7/10
----------
Train Loss: 0.0423 Acc: 0.9875
Val Loss: 0.2959 Acc: 0.9214
Epoch 8/10
----------
Train Loss: 0.1192 Acc: 0.9670
Val Loss: 0.1656 Acc: 0.9357
Epoch 9/10
----------
Train Loss: 0.0902 Acc: 0.9750
Val Loss: 1.3636 Acc: 0.7357
Epoch 10/10
----------
Train Loss: 0.0787 Acc: 0.9705
Val Loss: 0.2056 Acc: 0.9500


In [48]:
model.eval() 
all_preds = []
all_labels = []

# Evaluate the model on the test data
with torch.no_grad():
    for images, labels in dataloaders['test']:  # Use the test DataLoader
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted")
recall = recall_score(all_labels, all_preds, average="weighted")
f1 = f1_score(all_labels, all_preds, average="weighted")

# Print the metrics
print("Evaluation Metrics on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Evaluation Metrics on Test Data:
Accuracy: 0.8857
Precision: 0.8940
Recall: 0.8857
F1 Score: 0.8867


### Discussion
Provide a comparative analysis.

The pretrained model has demonstrated superior performance in terms of accuracy, precision, recall, and F1 score compared to model tuned through hyperparameter optimization.

In this part, the metrics achieved by the pretrained model—accuracy of 88.57%, precision of 89.40%, recall of 88.57%, and F1 score of 88.67%—indicate a strong and balanced performance. 

Given these results, it’s evident that the pretrained model’s inherent optimization outperforms hyperparameter-tuned models for this task. This highlights the advantage of leveraging pretrained weights for achieving high performance without extensive tuning.